In [ ]:
import src
import keras.backend as K
import os
import numpy as np
import sys
import re
import math
import io
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from  matplotlib.animation import FuncAnimation
from matplotlib import colors
from netCDF4 import Dataset
from IPython.display import clear_output
#data folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/ADL4CV_project/trainings')

sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/data')
#forces CPU usage
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #"" or "-1" for CPU, "0" for GPU
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
past = 2
name = f"sgan_{past}-1"

Load dataset.

In [ ]:
train, xval, test = src.load_datasets(past_frames=past)

In [ ]:
gan_train, gan_truth, gan_val, gan_val_truth, gan_test, gan_test_truth = src.split_datasets(
            train[:2000], xval, test, past_frames=past, augment=True)

Make discriminator labels.

In [ ]:
batch_size=16

In [ ]:
# Adversarial ground truths
real = -np.ones((batch_size, 1))
fake = np.ones((batch_size, 1))
#Generator ground truths
#real, fake = src.noisy_d_labels(real, fake)

Make generator but don't compile.

In [ ]:
generator = src.unet((64, 64, past), dropout=0, batchnorm=True, kernel_size=4, feature_mult=1)

In [ ]:
generator.summary()

Make discriminator and compile.

In [ ]:
discriminator = src.spatial_discriminator(condition_shape=(64, 64, past), dropout = 0.25, batchnorm=True, wgan=True)
discriminator.compile(loss=src.wasserstein_loss,
                      optimizer=keras.optimizers.RMSprop(lr=0.00005))

Inputs and outputs of the GAN.

In [ ]:
frame_t = keras.layers.Input(shape=(64, 64, past))

In [ ]:
generated = generator(frame_t)

In [ ]:
score = discriminator([frame_t, generated])

Freeze discriminator weights.

In [ ]:
discriminator.trainable = False
for l in discriminator.layers: l.trainable = False

Compile combined model.

In [ ]:
loss_weights=[0,1]

In [ ]:
combined = keras.models.Model(inputs=[frame_t], outputs=[generated, score])

In [ ]:
combined.compile(loss=[src.custom_loss(loss="l1"), src.wasserstein_loss],
                 optimizer=keras.optimizers.RMSprop(lr=0.00005),
                 loss_weights=loss_weights)

In [ ]:

log = {"g_loss":[],
       "d_loss":[],
       "g_metric":[],
       "d_metric":[],
       "d_loss_real":[],
       "d_loss_fake":[],
       "d_test_real":[],
       "d_test_fake":[]}

Train x epochs.

In [ ]:
epochs = 20000
# random seed
RND = 777
np.random.seed(RND)

In [ ]:
for epoch in range(epochs):
    
    if (epoch % 1000) < 25 or epoch % 500 == 0: # 25 times in 1000, every 500th
        d_iters = 100
    else:
        d_iters = 10
    for d_it in range(d_iters):
        discriminator.trainable = True
        for l in discriminator.layers: l.trainable = True
        idx = np.random.choice(gan_truth.shape[0], batch_size, replace=False)
        real_imgs = gan_truth[idx]
        training_batch = gan_train[idx]
        
        for l in discriminator.layers:
            weights = l.get_weights()
            weights = [np.clip(w, -0.01, 0.01) for w in weights]
            l.set_weights(weights)
        d_loss_real = discriminator.train_on_batch([training_batch, real_imgs], real)
        generated_imgs = generator.predict(training_batch) 
        d_loss_fake = discriminator.train_on_batch([training_batch, generated_imgs], fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    discriminator.trainable = False 
    for l in discriminator.layers: l.trainable = False
    
    idx = np.random.choice(gan_truth.shape[0], batch_size, replace=False)
    training_batch = gan_train[idx]
    training_truth = gan_truth[idx]
    
    g_loss = combined.train_on_batch(training_batch, [training_truth, real])
    
    #if g_loss[1] < 0.11 and loss_weights[0] > 2**(-4):
    #    loss_weights[0] /= 2
    #    combined.compile(loss=[src.custom_loss(loss="l1"), keras.losses.binary_crossentropy], optimizer=keras.optimizers.Adam(0.0002, 0.5),
    #             loss_weights=loss_weights)
    
    log["g_loss"].append(g_loss) #sum, obj, bce
    log["d_loss"].append(d_loss) #sum
    log["g_metric"].append(g_loss[1])
    #log["d_metric"].append(d_loss[1])
    log["d_loss_real"].append(d_loss_real)
    log["d_loss_fake"].append(d_loss_fake)
    
    print(f"\033[1m {epoch} [D loss: {d_loss}]\033[0m \n"+
          f"\033[1m {epoch} [G loss: {g_loss[0]}, G obj.: {g_loss[1]}, G wgan.: {g_loss[2]}]\033[0m \n"+
          f"\033[1m {epoch} [real loss: {d_loss_real}, fake loss: {d_loss_fake}]\033[0m")
    if (epoch%100 == 0 or epoch == epochs-1) and epoch > 0:
        src.sample_images(epoch, gan_test, gan_test_truth, past, generator)
        plot_training_curves(log, epoch, name, True)

In [ ]:
def plot_training_curves(log, epoch, name, wgan=False):
    total_g_loss = np.array(log["g_loss"])[:, 0]
    total_d_loss = np.array(log["d_loss"])[:, 0] if not wgan else np.array(log["d_loss"])
    smoothed_tgl = src.smooth(np.array(log["g_loss"])[:, 0])
    smoothed_tdl = src.smooth(np.array(log["d_loss"])[:, 0]) if not wgan else src.smooth(np.array(log["d_loss"]))
    objective_loss = np.array(log["g_loss"])[:, 1]

    f, (a0, a1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [5, 2]})
    a0.plot(total_g_loss, alpha=0.3, c="b")
    a0.plot(smoothed_tgl, c="b", label="generator")
    a0.grid()
    if wgan:
        a0.plot(np.array(log["d_loss_real"]), c="g", label="real")
        a0.plot(np.array(log["d_loss_fake"]), c="r", label="fake")
    else:
        a0.plot(total_d_loss, alpha=0.3, c="orange")
        a0.plot(smoothed_tdl, c="orange", label="discriminator")
    a0.legend()
    a1.plot(objective_loss, alpha=0.9, c="green", label="L1 objective")
    a1.grid()
    a1.legend()
    f.text(0.5, 0, 'Iterations', ha='center', va='center')
    f.text(0, 0.5, 'Loss', ha='center', va='center', rotation='vertical')

    f.tight_layout()
    f.savefig(f"Plots/{name}_epoch_{epoch}_curves.png")

## Save features

Save model history

In [ ]:
np.save(name+"_log",log)

Save model weights

In [ ]:
combined.save_weights(name+"_model.h5")

Load model and predict

In [ ]:
combined.load_weights("C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/ADL4CV_project/trainings/"+name+"/obj=1,ks=4/"+name+"_model.h5")

Predict future frames. Loads a 20 long sequence with 1000 sequence samples.

In [ ]:
sequence_test = src.load_datasets(prediction=True)

In [ ]:
sequence_test = src.augment_data(sequence_test[:100])

In [ ]:
sequence_test.shape

In [ ]:
#test
predictions = {}
past_frames = sequence_test[...,0:past]
test_truth = sequence_test[...,past:past+1]

In [ ]:
gen = combined.layers[1]

In [ ]:
for t in range(5):
    future = gen.predict(past_frames, batch_size=64)
    predictions[f"{t}"] = future
    past_frames = np.concatenate((past_frames[:,:,:,1:], predictions[f"{t}"]), axis=-1)
    test_truth = sequence_test[...,past+1+t:past+2+t]

Save example predictions

In [ ]:
def save_examples(name, test, predictions_dict, past, samples=0):
    fig, axs = plt.subplots(len(samples)*2,past+len(predictions_dict.keys()), figsize=(32, 32))
    fig.subplots_adjust(wspace=0.3, hspace=0.0)
    for n in range(len(samples)):
        vmax = np.max(test[n,:,:,:past])
        vmin = 0
        print(test.shape)
        for i in range(past):
            im = axs[2*n,i].imshow(test[samples[n], :,:,i], vmax=vmax,vmin=vmin)
            axs[2*n,i].axis('off')
            axs[2*n,i].set_title(f"Past frame {i+1}")
            src.colorbar(im)
            im = axs[2*n+1,i].imshow(test[samples[n], :,:,i], vmax=vmax,vmin=vmin)
            axs[2*n+1,i].axis('off')
            axs[2*n+1,i].set_title(f"Past frame {i+1}")
            src.colorbar(im)
        for i in range(past,past+len(predictions_dict.keys())):
            im = axs[2*n,i].imshow(predictions_dict[f"{i-past}"][samples[n], :,:,0], vmax=vmax, vmin=vmin)
            axs[2*n,i].axis('off')
            axs[2*n,i].set_title(f"Predicted frame {i-past+1}")
            src.colorbar(im)
            im = axs[2*n+1,i].imshow(test[samples[n], :,:,i], vmax=vmax, vmin=vmin)
            axs[2*n+1,i].axis('off')
            axs[2*n+1,i].set_title(f"Reference frame {i-past+1}")
            src.colorbar(im)
    fig.savefig(f"Plots/{name}_sequence_prediction.png")
    plt.close()


In [ ]:
save_examples(name, sequence_test, predictions, past, samples=[33,46]) # 33, 46, 54

In [ ]:
def save_examples(name, test, predictions_dict, past, samples=0):
    fig, axs = plt.subplots(len(samples)*2,past+4, figsize=(32, 32))
    fig.subplots_adjust(wspace=0.3, hspace=0.0)
    for n in range(len(samples)):
        vmax = np.max(test[n,:,:,:past])
        vmin = 0
        for i in range(past):
            im = axs[2*n,i].imshow(test[samples[n], :,:,i], vmax=vmax,vmin=vmin)
            axs[2*n,i].axis('off')
            axs[2*n,i].set_title(f"Past frame {i+1}")
            colorbar(im)
            im = axs[2*n+1,i].imshow(test[samples[n], :,:,i], vmax=vmax,vmin=vmin)
            axs[2*n+1,i].axis('off')
            axs[2*n+1,i].set_title(f"Past frame {i+1}")
            colorbar(im)
        for i in range(past,past+4):
            im = axs[2*n,i].imshow(predictions_dict[f"{i-past}"][samples[n], :,:,0], vmax=vmax, vmin=vmin)
            axs[2*n,i].axis('off')
            axs[2*n,i].set_title(f"Predicted frame {i-past+1}")
            colorbar(im)
            im = axs[2*n+1,i].imshow(test[samples[n], :,:,i], vmax=vmax, vmin=vmin)
            axs[2*n+1,i].axis('off')
            axs[2*n+1,i].set_title(f"Reference frame {i-past+1}")
            colorbar(im)
    fig.savefig(f"Plots/{name}_sequence_prediction.png")
    plt.close()

Calculate scores

In [ ]:
list(predictions.keys())

In [ ]:
norms = np.load(sys.path[0]+"/5min_long_pred_norms_compressed.npz")["arr_1"]

In [ ]:
# *4 bc of augmentaion (it concats the frames so the 0th 1000th 2000th and 3000th are the same sample just rotated)
test_norms = list(norms)*4

In [ ]:
#renormalize test samples
renormalized_test = np.array([sample * np.array(test_norms)[i] for i, sample in enumerate(sequence_test)])
renormalized_predictions = np.transpose((np.array([[sample * np.array(test_norms)[i] for i, sample in enumerate(predictions[key])] for key in predictions.keys()])[:,:,:,:,0]), (1,2,3,0))

In [ ]:
renormalized_predictions.shape

In [ ]:
#thresholds: 2, 8, 42
thresholds = [10, 50, 100]
scores = {}
for t in range(renormalized_predictions.shape[-1]): # loop over the predictions (4)
    for s in thresholds: # make a dict entry for each threshold score
        scores[f"pred_{t+1}_threshold_{s}"] = src.calculate_skill_scores(renormalized_predictions[...,t:t+1],
                                                                                     renormalized_test[...,past+t:past+1+t],
                                                                                     x=renormalized_test[...,past-1:past],
                                                                                     threshold=s)

In [ ]:
scores["pred_7_threshold_10"].keys()

In [ ]:
np.save(name+"_scores",scores)

In [ ]:
loaded_scores = np.load(sys.path[1]+"/"+name+"/"+name+"_scores.npy").item()

In [ ]:
np.mean((pd.Series(scores["pred_1_threshold_10"]["corr_to_input"]).dropna()))

In [ ]:
np.mean((pd.Series(scores["pred_1_threshold_10"]["corr_to_truth"]).dropna()))

In [ ]:
np.mean((pd.Series(scores["pred_1_threshold_100"]["corr_to_truth"]).dropna()))

In [ ]:
np.mean((pd.Series(scores["pred_1_threshold_100"]["corr_to_truth"]).dropna()))